<a href="https://colab.research.google.com/github/saddam2973/GEN-AI/blob/Assignments/Multi_RAG_Built.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U \langchain \langchain-community \langchain-text-splitters \unstructured \
python-docx \
python-pptx \
openpyxl \
pandas \
msoffcrypto-tool \
faiss-cpu \
sentence-transformers
!pip install -U langchain langchain-community langchain-core
!pip install rank_bm25 sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━

In [2]:
import os
import pandas as pd
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
    UnstructuredExcelLoader,
    DataFrameLoader
)

def load_documents(file_paths):
    all_docs = []

    for path in file_paths:
        ext = os.path.splitext(path)[1].lower()

        if ext == ".pdf":
            docs = PyPDFLoader(path).load()

        elif ext == ".txt":
            docs = TextLoader(path, encoding="utf-8").load()

        elif ext in [".doc", ".docx"]:
            docs = UnstructuredWordDocumentLoader(path).load()

        elif ext in [".ppt", ".pptx"]:
            docs = UnstructuredPowerPointLoader(path).load()

        elif ext in [".xls", ".xlsx"]:
            try:
                docs = UnstructuredExcelLoader(path).load()
            except:
                df = pd.read_excel(path)
                docs = DataFrameLoader(df, page_content_column=df.columns[0]).load()

        else:
            continue

        all_docs.extend(docs)

    return all_docs


In [3]:
file_paths = [
    "/content/Day 2.pdf",
    "/content/Interstellar Story.txt",
    "/content/Facial Expression Detection.docx",
    "/content/Intruder-Detection-System-using-.pptx",
    "/content/3_Logical_Functions.xlsx"
]

docs = load_documents(file_paths)

print("Documents loaded:", len(docs))


Documents loaded: 22


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = splitter.split_documents(docs)
print("Total chunks:", len(chunks))

Total chunks: 241


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


/tmp/ipython-input-4093892415.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")

Enter your Mistral API Key: ··········


In [7]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
from langchain_core.prompts import ChatPromptTemplate
import os

In [8]:
import langchain
print(langchain.__version__)


1.2.0


In [9]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain



prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:


{context}


Question: {input}
""")





In [11]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
import os
os.environ["MISTRAL_API_KEY"]

'QIS9Um5UE38FRDNlRP8VZcZkr8juPpCe'

In [18]:
rag_chain = create_retrieval_chain(
    retriever,
    document_chain
)

user_query = input("Enter Your Question : ")
response = rag_chain.invoke({"input": user_query})

print("\n ----THE RESPONSE IS----")
print(response["answer"])

Enter Your Question : What is Universe

 ----THE RESPONSE IS----
The term "Universe" is not directly mentioned in the provided context about the movie "Interstellar". However, in a broader sense, the universe in "Interstellar" refers to the vast expanse of space, encompassing multiple galaxies, stars, planets, and the wormhole that serves as the means for the intergalactic mission.
